In [5]:
import numpy as np
import pandas as pd
import os
import sys

from subprocess import call

# Using `partis` to generate synthetic datasets

We can use the software `partis` to generate synthetic datasets. (**Add link to partis**)

In [3]:
# configuration
partis_path = '/home/fede/src/partis_old'
output_path = '/home/fede/projects_local/davide/data/partis_RAW_v2/'
list_number_clones = [5000]
n_iter = 3

In [ ]:
# for i in [100,500,1500,3000]:
for i in list_number_clones:
    for j in range(0, n_iter):
        call('{1}/bin/partis simulate '
             '--parameter-dir {1}/test/reference-results/test/parameters/data/ '
             '--outfname {3}/clones_{0}.{2}.csv --n-sim-events {0} --n-leaves 200 '
             '--n-procs 16'.format(i, partis_path, j, output_path).split(' '))

This produces into `output_path` folder a list of RAW sequences.

To simplify the processing of IMGT/HighV-Quest, let's a unique `fasta` file where, in the `ID` string, there is also the identity of the original database name. This will allow us to recover our original databases splitted.

In [7]:
path = '/home/fede/projects_local/davide/data/partis_RAW_v2/'
files = [path + x for x in os.listdir(path) if x.endswith('.csv')]
num_files = 3

for i in range(num_files):
    with open(os.path.join(path, "all_{}_of_{}.fasta".format(i, num_files)), 'w') as f:
        for x in files[i::num_files]:
            df = pd.read_csv(x, index_col=None)
            # f.write(">" + x + "\n" + df['seq'][0] + "\n")
            for index, row in df.iterrows():
                f.write(">" + "_".join([x.split('/')[-1]] + [str(a) for a in row.values[:-8]]))
                f.write("\n")
                f.write(row['seq'])
                f.write("\n")

Ok! Now we can use IMGT to convert our `fasta` file(s) into databases which we can use as input to ICING.
To do so, connect to IMGT HighV-Quest software and upload the data.

When finished, an email will notify that results are ready.

In [ ]:
# run Changeo to convert IMGT into fasta file
# python MakeDb.py imgt -i <imgt output, zip or folder> -s <original fasta file> --scores

Divide now the IMGT-ChangeO processed files into a final list of databases which are usable from our method.

In [ ]:
path = '/home/fede/projects_local/davide/data/partis_RAW_v3/'
for f in [path + x for x in os.listdir(path) if x.endswith('db-pass.tab')]:
    df = pd.read_csv(f, dialect='excel-tab')
    # add the mut column
    df['MUT'] = (1 - df['V_IDENTITY']) * 100.
    
    df['DB'] = df['SEQUENCE_ID'].str.split('.csv').apply(lambda x: min(x, key=len))
    for i in df.DB.unique():
        df[df.DB == i].to_csv(path+str(i)+'.tab', index=False, sep='\t')